In [1]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

In [3]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import *

In [4]:
from pyspark.ml.feature import StopWordsRemover

In [5]:
dataset1 =spark.read.format('csv').options(header=True, inferSchema=True).load("dataset.csv")

In [6]:
df_en =dataset1.select("English")
df_en=df_en.dropna()

In [7]:
df_fr =dataset1.select("French")
df_fr=df_fr.dropna()

In [8]:
data_eng = df_en.select(col("English").alias("text"))
data_fr = df_fr.select(col("French").alias("text"))

In [9]:
df_en= data_eng.withColumn("label", lit(1))
df_fr = data_fr.withColumn("label", lit(0))

In [10]:
#on tokenise les deux ensembles
tokenizer_df_en = Tokenizer(inputCol="text", outputCol="text_token") 
token_df_en=tokenizer_df_en.transform(df_en)

In [11]:
tokenizer_df_fr = Tokenizer(inputCol="text", outputCol="text_token") 
token_df_fr=tokenizer_df_fr.transform(df_fr)

In [12]:
import nltk.corpus 
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword_en= stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
nltk.download('stopwords')
stopword_fr= stopwords.words('french')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
cleaned_df_en=StopWordsRemover(stopWords=stopword_en, inputCol="text_token", outputCol="text_cleaned")
df_en_cleaned=cleaned_df_en.transform(token_df_en)

In [15]:
cleaned_df_fr=StopWordsRemover(stopWords=stopword_fr,inputCol="text_token", outputCol="text_cleaned")
df_fr_cleaned=cleaned_df_fr.transform(token_df_fr)

In [16]:
train_fr, test_fr = df_fr_cleaned.randomSplit([0.8,0.2])
train_en, test_en = df_en_cleaned.randomSplit([0.8,0.2])

In [17]:
#concatener les 2 dataframes d'entrainement
train= train_fr.union(train_en)

In [18]:
#concatener les 2 dataframes de test
test= test_fr.union(test_en)

In [19]:
dataset= train.union(test)

In [20]:
dataset_final=dataset.drop("text_token")

In [22]:
dataset_final.write.mode('Overwrite').json("dataset_final.json")